### ANN  *version 1.20.04*
feedforward/backpropagation online

- `nx`: número de vectores de entrada
- `neuronEntrada`: número de neuronas en la acpa de entrada
- `neuronEscondida`: número de neuronas en la capa de escondida
- `neuronSalida`: número de neuronas en la capa de salida
- `alpha`: razón de aprendizaje
- `epoch`: número de epocas

In [28]:
from numba import jit
from numba import cuda
from numba import *
import numpy as np
import random as rm
from timeit import default_timer as timer

In [27]:
# Configuración de la red
nx = 10
neuronEntrada = 9
neuronEscondida = 10
neuronSalida = 10
alpha = 0.1

In [29]:
# Ingreso de datos
entrada = np.loadtxt('data/entrada.dat')
salida = np.loadtxt('data/salida.dat')

In [30]:
# Función de activación
def sigmoide(aux):
    return (1 + np.exp(-aux))**(-1)

def entrenamiento(epoch, nx, neuronEntrada, neuronEscondida, neuronSalida):
    #sort 
    sort = np.array(range(nx))

    # Feedforward
    a2 = np.zeros(neuronEscondida, dtype=float)
    a3 = np.zeros(neuronSalida, dtype=float)

    # Deltas
    d2 = np.zeros (neuronEscondida, dtype=float)
    d3 = np.zeros (neuronSalida, dtype=float)
    
    # Pesos
    omega1 = np.ones((neuronEntrada+1, neuronEscondida), dtype=float)
    omega2 = np.ones((neuronEscondida+1, neuronSalida), dtype=float)

    for i in range(neuronEntrada):
        for j in range(neuronEscondida):
            omega1[i,j] = rm.uniform(-1,1)

    for i in range(neuronEscondida):
        for j in range(neuronSalida):
            omega2[i,j] = rm.uniform(-1,1)

    start = timer()

    # Inicia entrenamiento
    for k in range(epoch):
        rm.shuffle(sort)

        for d in range(nx):
            Error = []
            e = 0

    # Capa E-H
            for i in range(neuronEscondida):
                a = 0

                for j in range(neuronEntrada):
                    a = (entrada[sort[d],j] * omega1[j,i]) + a

                a2[i] = sigmoide(a + omega1[neuronEntrada,i])

    # Capa H-S
            for i in range(neuronSalida):
                a = 0

                for j in range(neuronEscondida):
                    a = (a2[j] * omega2[j,i]) + a

                a3[i] = sigmoide(a + omega2[neuronEscondida, i])

    # Calculo de las Deltas
        # D3 S-H
            for i in range(neuronSalida):
                d3[i] = a3[i] * (1-a3[i]) * (-(salida[sort[d],i] - a3[i]))
        
        # D2 H-E
            for i in range(neuronEscondida):
                aux = 0

                for j in range(neuronSalida):
                    aux += omega2[i,j] * d3[j]
            
                d2[i] = a2[i] * (1 - a2[i]) * aux
    
    # Actualización de pesos
        # omega1 y bias

            for i in range(neuronEscondida):
                for j in range(neuronEntrada):
                    omega1[j,i] -= alpha * entrada[sort[d],j] * d2[i]

                omega1[neuronEntrada,i] -= alpha * d2[i]

        # omega2 y bias
            for i in range(neuronSalida):
                for j in range(neuronEscondida):
                    omega2[j,i] -= alpha * a2[j] * d3[i]

                omega2[neuronEscondida,i] -= alpha * d3[i]

    dt = timer() - start

    print('Entrenamiento concluido en %f s' % dt)

    return omega1, omega2

def feedforward(neuronEntrada, neuronEscondida, neuronSalida, x):
    # Feedforward
    a2 = np.zeros(neuronEscondida, dtype=float)
    a3 = np.zeros(neuronSalida, dtype=float)
    
    start = timer()

    for i in range(neuronEscondida):
        a = 0

        for j in range(neuronEntrada):
            a += x[j] * omega1[j,i]

        a2[i] = sigmoide(a + omega1[neuronEntrada, i])

    for i in range(neuronSalida):
        a = 0

        for j in range(neuronEscondida):
            a += a2[j] * omega2[j,i]

        a3[i] = sigmoide(a + omega2[neuronEscondida,i])

    dt = timer() - start

    print('Ejecución en %f s' % dt)
    print(a3)

In [31]:
# Función de activación
@jit
def jitSigmoide(aux):
    return (1 + np.exp(-aux))**(-1)

@jit
def jitEntrenamiento(epoch, nx, neuronEntrada, neuronEscondida, neuronSalida):
    
    #sort 
    sort = np.array(range(nx))

    # Feedforward
    a2 = np.zeros(neuronEscondida, dtype=float)
    a3 = np.zeros(neuronSalida, dtype=float)

    # Deltas
    d2 = np.zeros (neuronEscondida, dtype=float)
    d3 = np.zeros (neuronSalida, dtype=float)
    
    # Pesos
    omega1 = np.ones((neuronEntrada+1, neuronEscondida), dtype=float)
    omega2 = np.ones((neuronEscondida+1, neuronSalida), dtype=float)

    for i in range(neuronEntrada):
        for j in range(neuronEscondida):
            omega1[i,j] = rm.uniform(-1,1)

    for i in range(neuronEscondida):
        for j in range(neuronSalida):
            omega2[i,j] = rm.uniform(-1,1)

    start = timer()

    # Inicia entrenamiento
    for k in range(epoch):
        rm.shuffle(sort)

        for d in range(nx):
            Error = []
            e = 0

    # Capa E-H
            for i in range(neuronEscondida):
                a = 0

                for j in range(neuronEntrada):
                    a = (entrada[sort[d],j] * omega1[j,i]) + a

                a2[i] = jitSigmoide(a + omega1[neuronEntrada,i])

    # Capa H-S
            for i in range(neuronSalida):
                a = 0

                for j in range(neuronEscondida):
                    a = (a2[j] * omega2[j,i]) + a

                a3[i] = jitSigmoide(a + omega2[neuronEscondida, i])

    # Calculo de las Deltas
        # D3 S-H
            for i in range(neuronSalida):
                d3[i] = a3[i] * (1-a3[i]) * (-(salida[sort[d],i] - a3[i]))
        
        # D2 H-E
            for i in range(neuronEscondida):
                aux = 0

                for j in range(neuronSalida):
                    aux += omega2[i,j] * d3[j]
            
                d2[i] = a2[i] * (1 - a2[i]) * aux
    
    # Actualización de pesos
        # omega1 y bias

            for i in range(neuronEscondida):
                for j in range(neuronEntrada):
                    omega1[j,i] -= alpha * entrada[sort[d],j] * d2[i]

                omega1[neuronEntrada,i] -= alpha * d2[i]

        # omega2 y bias
            for i in range(neuronSalida):
                for j in range(neuronEscondida):
                    omega2[j,i] -= alpha * a2[j] * d3[i]

                omega2[neuronEscondida,i] -= alpha * d3[i]

    dt = timer() - start

    print('Entrenamiento concluido en %f s' % dt)

    return omega1, omega2

@jit
def jitFeedforward(neuronEntrada, neuronEscondida, neuronSalida, x):
    # Feedforward
    a2 = np.zeros(neuronEscondida, dtype=float)
    a3 = np.zeros(neuronSalida, dtype=float)
    
    start = timer()

    for i in range(neuronEscondida):
        a = 0

        for j in range(neuronEntrada):
            a += x[j] * omega1[j,i]

        a2[i] = jitSigmoide(a + omega1[neuronEntrada, i])

    for i in range(neuronSalida):
        a = 0

        for j in range(neuronEscondida):
            a += a2[j] * omega2[j,i]

        a3[i] = jitSigmoide(a + omega2[neuronEscondida,i])

    dt = timer() - start

    print('Ejecución en %f s' % dt)
    print(a3)

In [15]:
# Función de activación
@cuda.jit
def gpuSigmoide(aux):
    return (1 + np.exp(-aux))**(-1)

@cuda.jit
def gpuEntrenamiento(epoch, nx, neuronEntrada, neuronEscondida, neuronSalida):
    
    #sort 
    sort = np.array(range(nx))

    # Feedforward
    a2 = np.zeros(neuronEscondida, dtype=float)
    a3 = np.zeros(neuronSalida, dtype=float)

    # Deltas
    d2 = np.zeros (neuronEscondida, dtype=float)
    d3 = np.zeros (neuronSalida, dtype=float)
    
    # Pesos
    omega1 = np.ones((neuronEntrada+1, neuronEscondida), dtype=float)
    omega2 = np.ones((neuronEscondida+1, neuronSalida), dtype=float)

    for i in range(neuronEntrada):
        for j in range(neuronEscondida):
            omega1[i,j] = rm.uniform(-1,1)

    for i in range(neuronEscondida):
        for j in range(neuronSalida):
            omega2[i,j] = rm.uniform(-1,1)

    start = timer()

    # Inicia entrenamiento
    for k in range(epoch):
        rm.shuffle(sort)

        for d in range(nx):
            Error = []
            e = 0

    # Capa E-H
            for i in range(neuronEscondida):
                a = 0

                for j in range(neuronEntrada):
                    a = (entrada[sort[d],j] * omega1[j,i]) + a

                a2[i] = gpuSigmoide(a + omega1[neuronEntrada,i])

    # Capa H-S
            for i in range(neuronSalida):
                a = 0

                for j in range(neuronEscondida):
                    a = (a2[j] * omega2[j,i]) + a

                a3[i] = gpuSigmoide(a + omega2[neuronEscondida, i])

    # Calculo de las Deltas
        # D3 S-H
            for i in range(neuronSalida):
                d3[i] = a3[i] * (1-a3[i]) * (-(salida[sort[d],i] - a3[i]))
        
        # D2 H-E
            for i in range(neuronEscondida):
                aux = 0

                for j in range(neuronSalida):
                    aux += omega2[i,j] * d3[j]
            
                d2[i] = a2[i] * (1 - a2[i]) * aux
    
    # Actualización de pesos
        # omega1 y bias

            for i in range(neuronEscondida):
                for j in range(neuronEntrada):
                    omega1[j,i] -= alpha * entrada[sort[d],j] * d2[i]

                omega1[neuronEntrada,i] -= alpha * d2[i]

        # omega2 y bias
            for i in range(neuronSalida):
                for j in range(neuronEscondida):
                    omega2[j,i] -= alpha * a2[j] * d3[i]

                omega2[neuronEscondida,i] -= alpha * d3[i]

    dt = timer() - start

    print('Entrenamiento concluido en %f s' % dt)

    return omega1, omega2

@cuda.jit
def gpuFeedforward(neuronEntrada, neuronEscondida, neuronSalida, x):
    # Feedforward
    a2 = np.zeros(neuronEscondida, dtype=float)
    a3 = np.zeros(neuronSalida, dtype=float)
    
    start = timer()

    for i in range(neuronEscondida):
        a = 0

        for j in range(neuronEntrada):
            a += x[j] * omega1[j,i]

        a2[i] = gpuSigmoide(a + omega1[neuronEntrada, i])

    for i in range(neuronSalida):
        a = 0

        for j in range(neuronEscondida):
            a += a2[j] * omega2[j,i]

        a3[i] = gpuSigmoide(a + omega2[neuronEscondida,i])

    dt = timer() - start

    print('Ejecución en %f s' % dt)
    print(a3)

### Entrenamiento y verificación usando **intelpython**

In [39]:
# Entrenamiento
epoch = 1000
omega1, omega2 = entrenamiento(epoch, nx, neuronEntrada, neuronEscondida, neuronSalida)

Entrenamiento concluido en 2.552736 s


In [40]:
# Guardar weights entrenados
np.savetxt('weights/intelOmega1.dat',omega1,fmt='%.8e')
np.savetxt('weights/intelOmega2.dat',omega2,fmt='%.8e')

In [41]:
# Cargar pesos previamente entrenados
omega1 = np.loadtxt('weights/intelOmega1.dat')
omega2 = np.loadtxt('weights/intelOmega2.dat')

In [42]:
# Verificacion
x = np.array([1,1,1,1,0,1,1,1,1])

In [43]:
feedforward(neuronEntrada, neuronEscondida, neuronSalida, x)

Ejecución en 0.000326 s
[0.61778308 0.09398217 0.12924594 0.25233068 0.25828479 0.2370008
 0.3185424  0.17364377 0.32101041 0.14642757]


### Entrenamiento y verificación usando **NumbaCPU**

In [44]:
# Entrenamiento
epoch = 1000
omega1, omega2 = jitEntrenamiento(epoch, nx, neuronEntrada, neuronEscondida, neuronSalida)

Entrenamiento concluido en 2.812558 s


In [45]:
# Guardar weights entrenados
np.savetxt('weights/cpuOmega1.dat',omega1,fmt='%.8e')
np.savetxt('weights/cpuOmega2.dat',omega2,fmt='%.8e')

In [46]:
# Cargar pesos previamente entrenados
omega1 = np.loadtxt('weights/cpuOmega1.dat')
omega2 = np.loadtxt('weights/cpuOmega2.dat')

In [47]:
# Verificacion
x = np.array([1,1,1,1,0,1,1,1,1])

In [51]:
jitFeedforward(neuronEntrada, neuronEscondida, neuronSalida, x)

Ejecución en 0.000024 s
[0.670899   0.11756865 0.1237651  0.26594833 0.22135259 0.16553507
 0.42150847 0.12765484 0.33018993 0.23559897]


### Entrenamiento y verificación usando **NumbaGPU**